In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import os,datetime,warnings,time

import re

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Conv2DTranspose,Dropout,GRU,Activation,Add,Concatenate,BatchNormalization,Bidirectional,Embedding,Flatten

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
import string

warnings.filterwarnings('ignore')
sns.set()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def split_sentence(basephrase,L,F):
    b = basephrase.split(' ')
    X = []
    Y = []
    for k in range(len(b)-L-F+1):
        X.append(b[k:k+L])
        Y.append(b[k+L:k+L+F])
    return X,Y

In [3]:
def build_labeled_training_set(baselist,L=5,F=1,limited_load=-1):
    if limited_load>0: baselist = baselist[:limited_load]
    X = []
    Y = []
    for P in baselist:
        x,y = split_sentence(basephrase=P,L=L,F=F)
        for k in range(len(x)): 
            X.append(x[k])
            Y.append(y[k])   
    return X,Y

In [4]:
def encode_words_by_dict(L,encode_dict):
    Le = []
    for e in L:
        ll = [encode_dict.ix[k,'Code'] for k in e]
        Le.append(ll)
    return Le

In [5]:
def clean_dataset(X,Y,MAXLEN=15):
    indices = []
    for k in range(len(Y)):
        flag = False
        for yy in Y[k]: 
            if len(yy)>MAXLEN: flag = True
        if not flag: indices.append(k)
    X = [X[k] for k in indices]
    Y = [Y[k] for k in indices]
    return X,Y

In [6]:
bf = pd.read_csv('..\\..\\TF_data\\TURING\\twitter-sample\\train_tweets.csv',index_col='id')

In [7]:
df = bf[['tweet']].copy()

df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^a-zA-Z0-9_@ ]','',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\s+',' ',x.lower().strip()))

In [8]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user @user @user @user','@user',x))

df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user@user@user@user','@user',x))

In [9]:
df['tweet'] = df['tweet'].apply(lambda x: x+' <EOL>')

In [10]:
wordlist = list(set([kk for k in df['tweet'].tolist() for kk in k.split(' ')]))
encode_dict = pd.DataFrame(wordlist,columns=['Word']).reset_index().rename(columns={'index':'Code'}).set_index('Word')

In [11]:
encode_dict['Word'] = encode_dict.index
encode_dict.index = encode_dict.index.rename('idx')
encode_dict['Wordlength'] = encode_dict['Word'].apply(lambda x: len(x))

In [12]:
X,Y = build_labeled_training_set(baselist=df['tweet'].tolist())

In [13]:
X,Y = clean_dataset(X,Y)

In [14]:
Y0 = encode_words_by_dict(Y,encode_dict)

In [ ]:
ohe = OneHotEncoder()
Y1 = ohe.fit_transform(Y0)

In [85]:
MAXLEN = 15

x = [[ord(k) for k in XXX] for XX in X for XXX in XX]

X0 = [[[ord(x) for x in XXX] for XXX in XX] for XX in X]
X1 = [[x+([0]*(MAXLEN-len(x))) for x in XX] for XX in X0]

In [74]:
X1 = [np.array(x).transpose() for x in X0]

In [79]:
len([x for x in X1 if x.shape[0]!=15])

6554

In [86]:
[x for x in X0 if len(x)!=5]

[]